## 1. MNIST를 RNN으로

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [13]:
learning_rate= 0.001
total_epoch = 30
batch_size = 128

n_input = 28
n_step = 28
n_hidden = 128
n_class = 10

In [33]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [4]:
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

In [5]:
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [7]:
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]

In [8]:
model = tf.matmul(outputs, W) + b

In [9]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [19]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(total_epoch):
    total_cost= 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val
        
    print('Epoch:', '%04d' % (epoch + 1), 'Avg.cost =', '{:.3f}'.format(total_cost / total_batch))
    
print('최적화 완료!')

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

test_batch_size = len(mnist.test.images)

test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

print('정확도:', sess.run(accuracy, feed_dict={X: test_xs, Y: test_ys}))

Epoch: 0001 Avg.cost = 0.542
Epoch: 0002 Avg.cost = 0.240
Epoch: 0003 Avg.cost = 0.174
Epoch: 0004 Avg.cost = 0.154
Epoch: 0005 Avg.cost = 0.138
Epoch: 0006 Avg.cost = 0.118
Epoch: 0007 Avg.cost = 0.112
Epoch: 0008 Avg.cost = 0.106
Epoch: 0009 Avg.cost = 0.100
Epoch: 0010 Avg.cost = 0.098
Epoch: 0011 Avg.cost = 0.095
Epoch: 0012 Avg.cost = 0.085
Epoch: 0013 Avg.cost = 0.086
Epoch: 0014 Avg.cost = 0.085
Epoch: 0015 Avg.cost = 0.083
Epoch: 0016 Avg.cost = 0.079
Epoch: 0017 Avg.cost = 0.074
Epoch: 0018 Avg.cost = 0.078
Epoch: 0019 Avg.cost = 0.073
Epoch: 0020 Avg.cost = 0.068
Epoch: 0021 Avg.cost = 0.068
Epoch: 0022 Avg.cost = 0.066
Epoch: 0023 Avg.cost = 0.061
Epoch: 0024 Avg.cost = 0.064
Epoch: 0025 Avg.cost = 0.067
Epoch: 0026 Avg.cost = 0.058
Epoch: 0027 Avg.cost = 0.063
Epoch: 0028 Avg.cost = 0.069
Epoch: 0029 Avg.cost = 0.057
Epoch: 0030 Avg.cost = 0.061
최적화 완료!
정확도: 0.9732


## 10.2 단어 자동 완성

In [20]:
import numpy as np
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

In [28]:
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

In [21]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:
        input = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target)
        
    return input_batch, target_batch

learning_rate = 0.01
n_hidden = 128
total_epoch = 30
n_step =3
n_input = n_class = dic_len


In [ ]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [23]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

In [24]:
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

In [25]:
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

In [26]:
cost = tf.reduce_mean( tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [29]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})
    
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    
print('최적화 완료!')

Epoch: 0001 cost = 3.798557
Epoch: 0002 cost = 2.814528
Epoch: 0003 cost = 1.522980
Epoch: 0004 cost = 1.065668
Epoch: 0005 cost = 1.223558
Epoch: 0006 cost = 0.456911
Epoch: 0007 cost = 0.616175
Epoch: 0008 cost = 0.624520
Epoch: 0009 cost = 0.302506
Epoch: 0010 cost = 0.312327
Epoch: 0011 cost = 0.304988
Epoch: 0012 cost = 0.342797
Epoch: 0013 cost = 0.167950
Epoch: 0014 cost = 0.137598
Epoch: 0015 cost = 0.222229
Epoch: 0016 cost = 0.067474
Epoch: 0017 cost = 0.202642
Epoch: 0018 cost = 0.061743
Epoch: 0019 cost = 0.113613
Epoch: 0020 cost = 0.136710
Epoch: 0021 cost = 0.079752
Epoch: 0022 cost = 0.157222
Epoch: 0023 cost = 0.130370
Epoch: 0024 cost = 0.165374
Epoch: 0025 cost = 0.034489
Epoch: 0026 cost = 0.012479
Epoch: 0027 cost = 0.065858
Epoch: 0028 cost = 0.043340
Epoch: 0029 cost = 0.061239
Epoch: 0030 cost = 0.005861
최적화 완료!


In [30]:
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

In [31]:
input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy], feed_dict={X: input_batch, Y: target_batch})

In [32]:
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)
    
print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + '' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)


=== 예측 결과 ===
입력값: ['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov', 'kis', 'kin']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 1.0


## 3. Sequence to Sequence

In [1]:
import tensorflow as tf
import numpy as np

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

seq_data = [['word', '단어'], ['wood', '나무'], ['game', '놀이'], ['girl', '소녀'], ['kiss', '키스'], ['love', '사랑']]

1. 인코더 셀의 입력값을 위해 입력 단어를 한 글자씩 떼어 배열로 만듭니다.
2. 디코더 셀의 입력밧을 위해 출력 단어의 글자들을 배열로 만들고, 시작을 나타내는 심볼'S'를 맨 앞에 붙입니다.
3. 학습을 위해 비교할 디코더 셀의 출력값을 만들고, 출력의 끝을 알려주는 심볼 'E'를 마지막에 붙입니다.

In [3]:
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []
    
    for seq in seq_data:
        input = [num_dic[n] for n in seq[0]]
        output = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]
        
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)
        
    return input_batch, output_batch, target_batch

In [4]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 100

n_class = n_input = dic_len

In [5]:
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

In [6]:
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)
    
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    
    outputs, dec_state = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state=enc_states, dtype=tf.float32)

In [7]:
model = tf.layers.dense(outputs, n_class, activation=None)

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    
print('최적화 완료!')

Epoch: 0001 cost = 3.755240
Epoch: 0002 cost = 2.662431
Epoch: 0003 cost = 1.700984
Epoch: 0004 cost = 1.331005
Epoch: 0005 cost = 0.749661
Epoch: 0006 cost = 0.490378
Epoch: 0007 cost = 0.493383
Epoch: 0008 cost = 0.410356
Epoch: 0009 cost = 0.258217
Epoch: 0010 cost = 0.203878
Epoch: 0011 cost = 0.254099
Epoch: 0012 cost = 0.111028
Epoch: 0013 cost = 0.133420
Epoch: 0014 cost = 0.075950
Epoch: 0015 cost = 0.098040
Epoch: 0016 cost = 0.064622
Epoch: 0017 cost = 0.024987
Epoch: 0018 cost = 0.045631
Epoch: 0019 cost = 0.037159
Epoch: 0020 cost = 0.032225
Epoch: 0021 cost = 0.067716
Epoch: 0022 cost = 0.015288
Epoch: 0023 cost = 0.012642
Epoch: 0024 cost = 0.022679
Epoch: 0025 cost = 0.004493
Epoch: 0026 cost = 0.001641
Epoch: 0027 cost = 0.003600
Epoch: 0028 cost = 0.009993
Epoch: 0029 cost = 0.006258
Epoch: 0030 cost = 0.001460
Epoch: 0031 cost = 0.002373
Epoch: 0032 cost = 0.001007
Epoch: 0033 cost = 0.002274
Epoch: 0034 cost = 0.002631
Epoch: 0035 cost = 0.004762
Epoch: 0036 cost = 0

In [13]:
def translate(word):
    seq_data = [word, 'P' * len(word)]
    
    input_batch, output_batch, target_batch = make_batch([seq_data])

    prediction = tf.argmax(model, 2)

    result = sess.run(prediction, feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})

    decoded = [char_arr[i] for i in result[0]]

    end = decoded.index('E')
    translated = ''.join(decoded[:end])

    return translated

In [14]:
print('\n=== 번역 테스트 ===')

print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))


=== 번역 테스트 ===
word -> 단어
wodr -> 나무
love -> 사랑
loev -> 사랑
abcd -> 놀이사무
